# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.out** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **run your notebook and keep all running logs** so that we can check.

## 1 $n$-gram Language Model

In [1]:
!wget -O train.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/train.txt
!wget -O dev.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/dev.txt
!wget -O test.txt https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/lm/test.txt

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


### 1.1 Building vocabulary

#### Code

In [2]:
def load_data(dir): # dir: 'lm' or 'prep', mode: 'train' or 'dev'
    # load data
    import os

    if dir=='lm':
        data_dir = os.path.join(os.getcwd(), "data", "lm")
        train_path = os.path.join(data_dir, 'train.txt')
        dev_path = os.path.join(data_dir, 'dev.txt')
        test_path  = os.path.join(data_dir, 'test.txt')
    else :
        data_dir = os.path.join(os.getcwd(), "data", "prep")
        train_path = os.path.join(data_dir, 'dev.in')
        dev_path = os.path.join(data_dir, 'dev.in')
        test_path  = os.path.join(data_dir, 'test.in')

    with open(train_path, 'r', encoding="utf-8") as f:
        train = [l.strip() for l in f.readlines()]
    with open(dev_path, 'r', encoding="utf-8") as f:
        dev = [l.strip() for l in f.readlines()]
    with open(test_path, 'r', encoding="utf-8") as f:
        test = [l.strip() for l in f.readlines()]
    
    return train, dev, test

In [3]:
import nltk
def build_vocab(sentences, n):
    # add start-of-sentence tokens 〈s〉 and end-of-sentence〈/s〉 tokens.
    for sentence in sentences:
        if n > 1:
            sentence = "<s> "*(n-1) + sentence + " </s>"
        else:
            sentence = "<s>" + sentence + "</s>"
    
    # tokenize training data
    tokens = ' '.join(sentences).split(' ')
 
    # create a frequency list
    freq = nltk.FreqDist(tokens)

    # show the vocabulary size
    print(f'vocabulary size: {len(freq)}')

   # convert tokens that occur less than three times in the training data into a special unknown token
    return [("<UNK>" if freq[token] < 3 else token) for token in tokens]

In [4]:
# Test for the functions
n = 2
train, dev, test = load_data('lm')
train_tokens = build_vocab(train, n)
dev_tokens = build_vocab(dev, n)
test_tokens = build_vocab(test, n)

vocabulary size: 52640
vocabulary size: 19546
vocabulary size: 19936


#### Discussion

The vocab size of training set is 52640
The vocab size of dev set is 19546
The vocab size of test set is 19936

The number of parameters of the model is about the size of the training token.

### 1.2 $n$-gram Language Modeling

#### Code

In [5]:
def build_n_gram_model(tokens, n):
    if n==1: # unigram model
        freq_tokens = nltk.FreqDist(tokens)
        len_tokens = len(tokens)
        return {vocab: count/len_tokens for vocab, count in freq_tokens.items()}
    else: #bigram model or trigram model
        ngrams = nltk.ngrams(tokens, n)
        n_minus1_grams = nltk.ngrams(tokens, n-1)
        freq_ngrams = nltk.FreqDist(ngrams)
        freq_n_minus1_grams = nltk.FreqDist(n_minus1_grams)

        result = dict()
        for ngram, freq in freq_ngrams.items():
            freq_n_minus1 = freq_n_minus1_grams[ngram[:-1]]
            result.update({ngram: freq / freq_n_minus1})

        return result

def calculate_perplexity(model, test_tokens, n):
    import numpy as np
    test_ngrams = nltk.ngrams(test_tokens, n)

    result = 0
    for ngram in list(test_ngrams):
        if ngram in model:
            result += np.log(model[ngram])
    return np.exp(-1 * result / len(model))

In [6]:
n_gram_model = build_n_gram_model(train_tokens, n)
# print(n_gram_model)
train_perplexity = calculate_perplexity(n_gram_model, train_tokens, n)
print('train_perplexity: ', train_perplexity)
test_perplexity = calculate_perplexity(n_gram_model, test_tokens, n)
print('test_perplexity: ', test_perplexity)

train_perplexity:  26174205.990303412
test_perplexity:  6.1461403073513265


#### Discussion

For the Unigram, 

    the perplexity on training set:  1.0

    the perplexity on test test:  1.0

For the Bigram model, 

    the perplexity on training set:  26174205.990303412

    the perplexity on test test:  6.1461403073513265


### 1.3 Smoothing

#### 1.3.1 Add-one (Laplace) smoothing

##### Code

In [7]:
def build_add_one_n_gram_model(tokens, n):
    if n==1: # unigram model
        freq_tokens = nltk.FreqDist(tokens)
        len_tokens = len(tokens)
        return {vocab: count/len_tokens for vocab, count in freq_tokens.items()}
    else: #bigram model or trigram model
        ngrams = nltk.ngrams(tokens, n)
        n_minus1_grams = nltk.ngrams(tokens, n-1)
        freq_ngrams = nltk.FreqDist(ngrams)
        freq_n_minus1_grams = nltk.FreqDist(n_minus1_grams)

        result = dict()
        for ngram, freq in freq_ngrams.items():
            freq_n_minus1 = freq_n_minus1_grams[ngram[:-1]]
            result.update({ngram: (freq+1) / (freq_n_minus1+1)})

        return result

In [8]:
n_gram_model = build_add_one_n_gram_model(train_tokens, n)
# print(n_gram_model)
train_perplexity = calculate_perplexity(n_gram_model, train_tokens, n)
print('train_perplexity: ', train_perplexity)
test_perplexity = calculate_perplexity(n_gram_model, test_tokens, n)
print('test_perplexity: ', test_perplexity)

train_perplexity:  12863974.249825127
test_perplexity:  5.880734463739251


##### Discussion

For the training set, the perplexity of Bigram model is decreased from 26174205.990303412 to 12863974.249825127

For the test set, the perplexity of Bigram model is decreased from 6.1461403073513265 to 5.880734463739251

    It is shown that the perplexity of the model in both dataset decrease after implementing Add-one smoothing. 


##### Optional: Add-k smoothing

###### Code

###### Discussion

#### 1.3.2 Linear Interpolation

##### Code

In [9]:
def linear_interpolation(model1, model2, model3, lambda1, lambda2, lambda3):
    new_model = dict()

    for trigram, prob3 in model3.items():
        prob2 = model2[trigram[1:]]
        prob1 = model1[trigram[2]]
        new_model.update({trigram: lambda1*prob1 + lambda2*prob2 + lambda3*prob3})
    
    return new_model

In [10]:
# Test for the functions
unimodel = build_n_gram_model(train_tokens, 1)
bimodel = build_add_one_n_gram_model(train_tokens, 2)
trimodel = build_add_one_n_gram_model(train_tokens, 3)

lambda1 = 0
lambda2 = 1/3
lambda3 = 2/3

linear_interpolation_model = linear_interpolation(unimodel, bimodel, trimodel, lambda1, lambda2, lambda3)
train_perplexity = calculate_perplexity(linear_interpolation_model, train_tokens, n)
print('train_perplexity: ', train_perplexity)
test_perplexity = calculate_perplexity(linear_interpolation_model, test_tokens, n)
print('test_perplexity: ', test_perplexity)

train_perplexity:  1.0
test_perplexity:  1.0


##### Discussion

##### Optional: Optimization

###### Discussion

## 2 Preposition Prediction

In [11]:
!wget -O dev.in https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.in
!wget -O dev.out https://raw.githubusercontent.com/ranpox/comp7607-fall2022/main/assignments/A1/data/prep/dev.out

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [12]:
train, dev, test = load_data('prep')

dev_tokens = build_vocab(dev, n)
test_tokens = build_vocab(test, n)



vocabulary size: 7312
vocabulary size: 7310
